# I See Dead People: An Analysis

Bryan Bumgardner, Data Scientist  
February - March 2016

### Death is a fact of life.   
That being said, it's worth studying how it fits in our popular culture. During the Metis Data Science bootcamp (shameless plug), while working on another project I discovered bodycounters.com[http://www.bodycounters.com/]. The dedicated volunteers at this site count and categorize the number of deaths in movies, and to date, have counted for over 2,000 movies. Go grab some beers, count the dead people in your favorite movie, and contribute to the site. 

The data are sitting on their site for anyone to see, and it gave me an idea. I reached out to the site and had some correspondence with Dana, who was gracious enough to share a CSV of all the data. I then cross referenced this data with information from another site, thenumbers.com, which shares basic budget and financial information about thousands of movies. I took this data, mashed it together, and asked some questions. 

Thanks again to the volunteers at bodycounters. Y'all are the greatest. Below is a quick data science passion project to analyze and find patterns in this great data you've collected. 

### Major questions:
1. Is the amount of on-screen deaths going up in modern movies?
2. Does a movie's death count have an effect on that movie's domestic gross?
3. Does a movie's death count have an effect on that movie's rating? 
4. How does budget factor into a movie's death count?
5. Is there a relationship between the movie's MPAA rating and body count?


### Table of Contents:
1. Reading and combining the data
2. Outputting various data frames
3. Visualizing
4. Conclusions
5. Documentation of the data

In [4]:
import pandas as pd 
import numpy as np
from patsy import dmatrices

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression 

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

%matplotlib inline
sns.set(color_codes=True)

### Step 1: read in and combine the data

Rather than building a web scraper, I elected to use import.io to get financial data from the-numbers. Their site was not so cooperative but the data is good. Some processing was done with Excel before this import.


### 5. Documentation of the data:

Budget and financial data of movies: http://www.the-numbers.com/movie/budgets/all